In [1]:
!pip install transformers rake-nltk nltk sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 25.0 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=d2ce63c175f1d474ad902628c914f95eabe9676ed15b3f73d31461dbee7204b9
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=f4fa8b0b5611ed1de284c0fedf72c1f10b1f7ecaf9e679b7043880b8317cff11
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [2]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import nltk
import re
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer as SumyTokenizer
from sumy.summarizers.lsa import LsaSummarizer
from rake_nltk import Rake
from transformers import BertTokenizer

# Скрипт для очищення коментаря

переробити під інпут тексту

In [4]:
name_df = "test/test"

In [5]:
# Завантаження даних
test_labels_df = pd.read_csv('/content/drive/MyDrive/data/test_labels/test_labels.csv')
test_df_path = f'/content/drive/MyDrive/data/{name_df}.csv'
test_df = pd.read_csv(test_df_path)

# Об'єднання датафреймів та видалення рядків з -1
merged_df = pd.merge(test_df, test_labels_df, on='id')
merged_df = merged_df[~(merged_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']] == -1).any(axis=1)]
merged_df.shape

(63978, 8)

In [6]:

# Завантажте токенізатор punkt
nltk.download('punkt')

# Завантаження стоп-слів для RAKE
nltk.download('stopwords')

# Ініціалізація токенайзера BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Функція для очищення тексту від цифр, знаків і спеціальних символів, крім ком і крапок
def clean_text_advanced(text):

    # Приводимо текст до нижнього регістру
    text = text.lower()
    # видаляємо все що не англ.
    text = re.sub(r"[^a-zA-Z.,\s]", "", text)

    # Видаляємо всі цифри
    text = re.sub(r'\d+', '', text)

    # Видаляємо символи пунктуації, окрім коми і крапки
    text = re.sub(r'[^\w\s,.]', '', text)

    # Видаляємо зайві символи нового рядка і замінюємо їх на пробіли
    text = text.replace('\n', ' ')

    # Видаляємо зайві пробіли
    text = re.sub(r'\s+', ' ', text).strip()

    # Видаляємо слова, довжина яких більше 40 символів
    text =  re.sub(r'\S{40,}', '', text)

    # Регулярний вираз для знаходження посилань
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_pattern, '', text)

    # Видалення фраз, що повторюються більше двох разів підряд
    text = re.sub(r'(\b\w+\b)(\s+\1)+', r'\1', text)

    # Видалення повторюваних речень більше ніж два рази підряд
    text = re.sub(r'(\b.+?\b)(\s*\1)+', r'\1', text)

   # Видалення повторюваних однакових літер (наприклад, AAAAAAA)
    text = re.sub(r'(.)\1{2,}', r'\1', text)  # Заміна 3 або більше однакових букв

    # Видалення повторюваних шаблонів, де кожна друга літера інша (наприклад, XAXAXA)
    text = re.sub(r'(.)(.)\1\2{2,}', r'\1\2', text)  # Заміна патернів повторів XAXAXA, XAXAXXAXX

    # Видаляємо повторювані послідовності сміху (ahahah, muahahah)
    text = re.sub(r'(ha|ah|mu|lol){2,}', '', text)

    # Видаляємо повторювані фрази більше ніж двічі (наприклад, "i hate you" або "die die die")
    text = re.sub(r'(\b\w+\b)(\s+\1){2,}', r'\1', text)

    # Видаляємо повторення фраз (двох і більше разів)
    text = re.sub(r'\b(\w+\b(?:\s+\w+\b){0,5})\s*(\1\s*)+', r'\1', text)

    # Видаляємо довгі повторення одного і того ж слова (більше 2 разів)
    text = re.sub(r'\b(\w+)\s+\1\s+\1(?:\s+\1)+', r'\1', text)

    # Видаляємо довгі рядки сміху (ahahaha, lol, muahaha) - залишаємо не більше 1-2 повторів
    text = re.sub(r'(\b(?:ha|ah|lol|mu|muah)+\b(?:\s*\b(?:ha|ah|lol|mu|muah)+\b){2,})', r'\1', text)

    return text




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# клауд скорочення коментаря

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import numpy as np

test_claud = merged_df

class TextSummarizer:
    def __init__(self):
        # Завантажуємо необхідні ресурси NLTK
        nltk.download('punkt')
        nltk.download('stopwords')
        self.stop_words = stopwords.words('english')

    def preprocess_text(self, text):
        # Розбиваємо текст на речення
        sentences = sent_tokenize(text)

        # Токенізація та видалення стоп-слів
        word_tokens = []
        for sentence in sentences:
            words = word_tokenize(sentence.lower())
            words = [w for w in words if w.isalnum() and w not in self.stop_words]
            word_tokens.extend(words)

        return sentences, word_tokens

    def get_sentence_scores(self, sentences, word_tokens):
        # Створюємо частотний розподіл слів
        freq_dist = FreqDist(word_tokens)

        # Обчислюємо оцінки для кожного речення
        sentence_scores = {}
        for i, sentence in enumerate(sentences):
            words = word_tokenize(sentence.lower())
            words = [w for w in words if w.isalnum()]
            score = sum([freq_dist[word] for word in words if word not in self.stop_words])
            sentence_scores[i] = score / len(words) if words else 0

        return sentence_scores

    def summarize(self, text, target_length=512):

        # видаляємо зайве з коментаря
        text = clean_text_advanced(text)


        # Попередня обробка тексту
        sentences, word_tokens = self.preprocess_text(text)

        # Отримуємо оцінки речень
        sentence_scores = self.get_sentence_scores(sentences, word_tokens)

        # Сортуємо речення за оцінками
        sorted_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)

        # Вибираємо найважливіші речення
        selected_sentences = []
        current_length = 0

        for sentence_idx, _ in sorted_sentences:
            sentence = sentences[sentence_idx]
            sentence_length = len(sentence)

            if current_length + sentence_length <= target_length:
                selected_sentences.append((sentence_idx, sentence))
                current_length += sentence_length
            else:
                break

        # Відновлюємо оригінальний порядок речень
        summary = ' '.join([sentence for _, sentence in sorted(selected_sentences)])

        return summary

# Приклад використання
summarizer = TextSummarizer()

# Застосовуємо метод summarize до кожного коментаря у колонці 'comment_text'
test_claud['comment_text'] = test_claud['comment_text'].apply(lambda x: summarizer.summarize(x, target_length=512))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
test_claud.to_csv('/content/drive/MyDrive/data/test_claud.csv', index=False)

In [25]:
# Вибір перших 100 рядків з merged_df
sample_df = test_claud.head(1000)


In [26]:
sample_df.shape

(1000, 8)

In [3]:
data = pd.read_csv('/content/drive/MyDrive/data/test_claud.csv')

In [12]:
# Видаляємо рядки з відсутніми коментарями
data = data.dropna(subset=['comment_text'])

# Перевірка кількості рядків після видалення
print(f"Залишилося рядків: {len(data)}")


Залишилося рядків: 63477


In [16]:
# Підготовка даних: тексти і мітки (labels)
texts = data['comment_text'].values
labels = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values


Using device: cuda


<ipython-input-16-b6dd8169b155>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)
/usr/local/lib/python3.10/dist-packages

Accuracy: 0.8662
Precision: 0.5672
Recall: 0.6535
F1 Score: 0.5890


# для моделі з розширенням .pt

In [17]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Перевірка доступності GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Шлях до моделі
model_path = "/content/drive/My Drive/bert_toxicity_full_model.pt"

# Перевірка існування файлу
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}")

# Завантаження моделі
try:
    # Спроба завантажити як повну модель
    model = torch.load(model_path, map_location=device)
except:
    try:
        # Спроба завантажити як state_dict
        model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)
        state_dict = torch.load(model_path, map_location=device)
        model.load_state_dict(state_dict)
    except Exception as e:
        print(f"Error loading model: {e}")
        print("Trying to load model without strict parameter...")
        try:
            # Спроба завантажити без строгої відповідності ключів
            model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)
            state_dict = torch.load(model_path, map_location=device)
            model.load_state_dict(state_dict, strict=False)
        except Exception as e:
            raise Exception(f"Failed to load model: {e}")

model.to(device)
model.eval()

# Завантаження токенізатора
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Підготовка даних
max_length = 128

def prepare_data(texts):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    return encodings

# Створення DataLoader
batch_size = 32
encodings = prepare_data(texts)
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels, dtype=torch.float))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Функція для отримання передбачень
def get_predictions(model, dataloader):
    predictions = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids, attention_mask, _ = [b.to(device) for b in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions.append(torch.sigmoid(logits).cpu().numpy())
    return np.vstack(predictions)

# Отримання передбачень
predictions = get_predictions(model, dataloader)

# Перетворення ймовірностей у бінарні мітки
binary_predictions = (predictions > 0.5).astype(int)

# Функція оцінки результатів для мульти-лейбл класифікації
def evaluate_multilabel_results(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='micro')
    recall = recall_score(true_labels, predicted_labels, average='micro')
    f1 = f1_score(true_labels, predicted_labels, average='micro')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Оцінка по кожному класу
    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    for i, class_name in enumerate(class_names):
        precision = precision_score(true_labels[:, i], predicted_labels[:, i])
        recall = recall_score(true_labels[:, i], predicted_labels[:, i])
        f1 = f1_score(true_labels[:, i], predicted_labels[:, i])
        print(f"\n{class_name}:")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")

# Оцінка результатів
evaluate_multilabel_results(labels, binary_predictions)

Using device: cuda


<ipython-input-17-e3f1d303b192>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)
/usr/local/lib/python3.10/dist-packages

Accuracy: 0.8662
Precision: 0.5579
Recall: 0.8126
F1 Score: 0.6616

toxic:
Precision: 0.5238
Recall: 0.9056
F1 Score: 0.6637

severe_toxic:
Precision: 0.3252
Recall: 0.4793
F1 Score: 0.3875

obscene:
Precision: 0.6208
Recall: 0.7732
F1 Score: 0.6887

threat:
Precision: 0.6207
Recall: 0.4306
F1 Score: 0.5085

insult:
Precision: 0.5783
Recall: 0.8083
F1 Score: 0.6743

identity_hate:
Precision: 0.7341
Recall: 0.5241
F1 Score: 0.6116


# для моделі з розширенням .pth

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Перевірка доступності GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Шлях до моделі
model_path = "/content/drive/MyDrive/toxicity_model/toxicity_model.pth"

# Перевірка існування файлу
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}")

# Завантаження моделі
try:
    # Спочатку створюємо модель з архітектурою BERT
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

    # Завантажуємо стан моделі
    state_dict = torch.load(model_path, map_location=device)

    # Перевіряємо, чи завантажений об'єкт є словником стану
    if isinstance(state_dict, dict):
        # Якщо це словник стану
        if 'state_dict' in state_dict:
            # Якщо стан знаходиться під ключем 'state_dict'
            model.load_state_dict(state_dict['state_dict'])
        else:
            # Якщо це просто словник стану
            model.load_state_dict(state_dict)
    elif isinstance(state_dict, BertForSequenceClassification):
        # Якщо це повна модель
        model = state_dict
    else:
        raise TypeError("Unexpected type of loaded object")

except Exception as e:
    raise Exception(f"Failed to load model: {e}")

model.to(device)
model.eval()

# Завантаження токенізатора
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Підготовка даних
max_length = 128

def prepare_data(texts):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    return encodings

# Створення DataLoader
batch_size = 32
encodings = prepare_data(texts)
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels, dtype=torch.float))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Функція для отримання передбачень
def get_predictions(model, dataloader):
    predictions = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids, attention_mask, _ = [b.to(device) for b in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions.append(torch.sigmoid(logits).cpu().numpy())
    return np.vstack(predictions)

# Отримання передбачень
predictions = get_predictions(model, dataloader)

# Перетворення ймовірностей у бінарні мітки
binary_predictions = (predictions > 0.5).astype(int)

# Функція оцінки результатів для мульти-лейбл класифікації
def evaluate_multilabel_results(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='micro')
    recall = recall_score(true_labels, predicted_labels, average='micro')
    f1 = f1_score(true_labels, predicted_labels, average='micro')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Оцінка по кожному класу
    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    for i, class_name in enumerate(class_names):
        precision = precision_score(true_labels[:, i], predicted_labels[:, i])
        recall = recall_score(true_labels[:, i], predicted_labels[:, i])
        f1 = f1_score(true_labels[:, i], predicted_labels[:, i])
        print(f"\n{class_name}:")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")

# Оцінка результатів
evaluate_multilabel_results(labels, binary_predictions)

Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-19-3cacb65a84ed>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We rec

In [10]:
import torch
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification

# Шлях до збереженої моделі
MODEL_PATH = '/content/drive/My Drive/toxicity_model/toxicity_model.pth'

In [ ]:
# Підготовка даних
texts = data['comment_text'].values
labels = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
print("Дані підготовлені")

# Клас для створення набору даних
class ToxicityDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(label)
        }

# Визначення пристрою - тільки CPU
device = torch.device('cpu')
print(f"Використовується пристрій: {device}")

# Завантаження збережених ваг моделі
state_dict = torch.load(MODEL_PATH, map_location=device)

# Ініціалізація моделі з правильною кількістю міток
num_labels = len(state_dict['classifier.weight'])
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Завантаження збережених ваг у модель
model.load_state_dict(state_dict)
model.to(device)
model.eval()

# Ініціалізація токенізатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Створення тестового набору даних та завантажувача
test_dataset = ToxicityDataset(texts, labels, tokenizer, max_len=256)
test_loader = DataLoader(test_dataset, batch_size=16)

# Функція для отримання прогнозів
def get_predictions(model, data_loader):
    model.eval()
    predictions = []
    real_values = []
    with torch.no_grad():
        for batch in data_loader:
            inputs = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(inputs, attention_mask=attention_mask)
            preds = torch.sigmoid(outputs.logits)  # Сигмоїд для мульти-лейбл задач
            preds = (preds > 0.5).int()  # Встановлюємо поріг

            predictions.append(preds)
            real_values.append(labels)

    predictions = torch.cat(predictions).numpy()
    real_values = torch.cat(real_values).numpy()
    return predictions, real_values

# Отримання прогнозів
print("Початок прогнозування...")
y_pred, y_true = get_predictions(model, test_loader)
print("Прогнозування завершено")

# Виведення результатів
print("\nРезультати класифікації:")
print(classification_report(y_true, y_pred, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], zero_division=0))

# Додатково: обчислення загальної точності
accuracy = accuracy_score(y_true.flatten(), y_pred.flatten())
print(f"\nЗагальна точність: {accuracy:.4f}")


Дані підготовлені
Використовується пристрій: cpu


<ipython-input-11-7ac8428be9db>:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(MODEL_PATH, map_location=device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Початок прогнозування...
